# Import Libraries


In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import tensorflow as tf
import math
import os
import datetime

# tf.enable_eager_execution()

import time



# Definition of Input parameters

In [2]:
N_ref = 2

N_time_steps = 1600 * N_ref
run_name = f'exp_check{int(time.time())}'

name_fine = 'exitq.txt'

y_t = np.loadtxt(name_fine,dtype='float32')

Num_fine = y_t.shape[0]
Num_coarse = 64

# T_8 = 1.6* math.pi  

Num_dx = 21

dt = 54 / N_ref
dx = 1000.0/(Num_dx-1)


T_8 = (N_time_steps)*dt 
os.mkdir(run_name)

In [3]:
T_8


86400.0

In [4]:
len(y_t[0])

1600

# Functions for Simulator

In [5]:
def simple_conv(x,filter_laplace):
    conv = tf.nn.conv1d(x,filter_laplace,1,'SAME')
    return conv

def laplace_1D(x,dt,dx):

    a= dt/(dx ** 2.0)
    filter_laplace = tf.constant([[[1.0*a]],[[-2.0*a]],[[1.0*a]]])

    return simple_conv(x,filter_laplace)

def next_step(P_old, K,dt,dx):
  
    div_P = laplace_1D(P_old,dt,dx)
    P_new = P_old + K * div_P 

    return P_new

def make_bound(P_new,t):

    P_boundary_add = np.zeros([Num_coarse,Num_dx,1],dtype='float32')
    P_boundary_add[:,0,0] = left_bound(t)

    P_boundary_mul = np.ones([Num_coarse,Num_dx,1],dtype='float32')
    P_boundary_mul[:,[0,-1],0] = 0

    P_new = P_new * tf.constant(P_boundary_mul) +tf.constant(P_boundary_add)

    return(P_new)
  
def left_bound(t):
  
    Dh = 1
    A = 1e-4
    B = 1 
    H =  Dh * (B + math.exp(-A*t))

    return H



# Functions for Plotting 

In [6]:
def plot(Q_final,q_final,y_t,Index_training,T_,run_name):
    
    #     print(q_final)
    b= -int(np.log10(T_[0])//1)
    name = './'+run_name+'/cdf_time_'+ str(Index_training)+'_level_em'+str(b)
    y_ = y_t[:,Index_training,None]
    Q_ind = np.unravel_index(np.argsort(Q_final,axis=0),Q_final.shape)
    
    Q_sort = Q_final[Q_ind[0]]
    q_sort  = q_final[Q_ind[0]]
    q_cum = np.cumsum(q_sort)
    
    y_ind = np.unravel_index(np.argsort(y_,axis=0),y_.shape)
    u_ = y_[y_ind]   
    x = np.linspace(0, 1, Num_fine)
    plt.plot( u_,x);
    
    print(name);
    
    plt.plot( Q_sort[:,0,0,0,0],q_cum);

    plt.xlabel('Q')
    plt.ylabel('Probability')
    plt.grid(True)

    plt.savefig(name, dpi=150)
    plt.close()

# Functions for Loss calculation

In [7]:
def dw(Y,Q,q,T):
    
    dw = tf.zeros([Num_coarse,Num_fine])
    print(Q[0].shape)
    for i in range(counter//N_ref):
        
        Y_in = tf.reshape(Y[:,i],[-1,1])
        Q_ = Q[i]
        dw += tf.squared_difference(tf.transpose(Y_in),Q_[:,:,0])/tf.nn.moments(Y_in,axes=[0])[1] 

    return dw

def F(q,dw,T):
    q = tf.reshape(q,[-1,1])
    dw_min = tf.reduce_min(dw,0,keepdims=True)
    dw_m_min = -dw + dw_min
    
    F = q * tf.exp(dw_m_min/T)
    F = tf.reduce_sum(F,0,keep_dims=True)
       
    F = tf.log(F)- dw_min/T
    F = - tf.reduce_sum(F)/Num_fine
    
    return F

def find_max_dic(Q,Dis_q):
    Q_srt_min = 0.0

    for i in range(counter//N_ref):
        Q_srt = np.resize(np.sort(Q[i][0:Dis_q+1]),[Dis_q])      
        Q_srt = np.min(np.abs(np.ediff1d(Q_srt)))
        Q_srt_min = np.max([Q_srt_min,Q_srt])
    return Q_srt_min

# Functions for probability updates

In [8]:
def p_y_X_f(dw_,q,T,p_y_X,Dis_q):   
    dw_ = dw_[0:Dis_q,:]

    q_mat = tf.reshape(q[0:Dis_q],[-1,1]) * tf.ones([1,Num_fine])

    dw_min = tf.reduce_min(dw_, axis=0,keepdims=True)
    p_y_X_ = q_mat * tf.exp(-(dw_-dw_min)/T)

    p_y_X_sum = tf.reduce_sum(p_y_X_, axis=0,keepdims=True)

    p_y_X_ = p_y_X_ / p_y_X_sum

    New_p_y_X = tf.assign(p_y_X[0:Dis_q,:],p_y_X_)
    return New_p_y_X


def q_f(p_y_X_,q_val,eps=1e-10):

    eps = 2 * eps/Num_coarse
    q_ = tf.reduce_mean(p_y_X_, axis=1)

    New_q = tf.assign(q_val,tf.log(q_+eps))
    return New_q

def Update_Dis(Dis_q,q_val,K_log):
    
    New_q_2 = []
    New_q_2.append(tf.assign(q_val[Dis_q:2*Dis_q],q_val[0:Dis_q]))

    for i in range(counter//N_ref):       
        a = tf.random_normal([Dis_q])
        New_q_2.append(tf.assign(K_log[i][Dis_q:2*Dis_q],K_log[i][0:Dis_q])*(1+0.0001 * a))
        New_q_2.append(tf.assign(K_log[i][0:Dis_q],K_log[i][0:Dis_q])*(1-0.0001 * a))
    return New_q_2

def MI_f(dw,p_y_X,T,q):
#     print(dw)
    dw = dw - tf.reduce_min(dw, axis=0)
#     print(tf.reduce_min(dw, axis=0))
    MI = dw/T - tf.log(tf.reduce_sum(tf.reshape(q,[-1,1]) * tf.exp(-dw/T) , axis=0,keepdims=True))
    MI = tf.reduce_mean(MI * p_y_X)
    return MI

# Initialization of Pressure field

In [9]:
P_initial = tf.Variable(tf.zeros([Num_coarse,Num_dx,1])+2-2/1000*tf.reshape(tf.range(0,1000+dx,dx),shape=[1,Num_dx,1]),trainable=False)

P_boundary_add = np.zeros([Num_coarse,Num_dx,1],dtype='float32')
P_boundary_add[:,0,0] = left_bound(0)

P_boundary_mul = np.ones([Num_coarse,Num_dx,1],dtype='float32')
P_boundary_mul[:,[0,-1],0] = 0

P_initial = P_initial * tf.constant(P_boundary_mul) +tf.constant(P_boundary_add)

P_initial = make_bound(P_initial,0)

# initialize Run

In [10]:
# K_mean = np.load('K_mean_final.npy')
K_mean = np.ones((N_time_steps,1,1,1))*1e-5
t = 0 
counter = 0
P_old = P_initial 

pressure = []
K_log = []
K_loss= []
Q_ = []

dh_check = []

# initialize prob
q_val_np = np.zeros([Num_coarse])
q_val_np[0] = np.log(Num_coarse*1e9)
q_val = tf.Variable(q_val_np,name='q_val',dtype='float32',trainable=True)
q = tf.nn.softmax(q_val)

p_y_X = tf.Variable(tf.zeros([Num_coarse,Num_fine]),trainable=False)

S = 1e-6
while counter<N_time_steps:

    t = t + dt
    
    if counter%N_ref == 0:
        log_K = tf.Variable((tf.ones([Num_coarse,1,1]) + tf.random_normal([Num_coarse,1,1]) *0.01 )*(np.log(K_mean[counter])) )
        K = tf.exp(log_K)

        K_loss.append(K)
        K_log.append(log_K)


    P_new = next_step(P_old, K/S, dt, dx)
    P_new = make_bound(P_new,t)
    
    if counter%N_ref == 0:

        dh = tf.reshape((P_old[:,Num_dx//2-1,0] - P_old[:,Num_dx//2+1,0])/(2*dx),[Num_coarse,1,1]) 
        Q = dh * K
        
        dh_check.append(dh)
        pressure.append(P_new)  
        Q_.append(Q)
        
        
        print(counter)
    counter +=1
    P_old = P_new


0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116
118
120
122
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200
202
204
206
208
210
212
214
216
218
220
222
224
226
228
230
232
234
236
238
240
242
244
246
248
250
252
254
256
258
260
262
264
266
268
270
272
274
276
278
280
282
284
286
288
290
292
294
296
298
300
302
304
306
308
310
312
314
316
318
320
322
324
326
328
330
332
334
336
338
340
342
344
346
348
350
352
354
356
358
360
362
364
366
368
370
372
374
376
378
380
382
384
386
388
390
392
394
396
398
400
402
404
406
408
410
412
414
416
418
420
422
424
426
428
430
432
434
436
438
440
442
444
446
448
450
452
454
456
458
460
462
464
466
468
470
472
474
476
478
480
482
484
486
488
490
492
494
496
498
500
502
504
506
508
510
512
514
516
518
520
522
524
526

In [11]:
y_t.shape

(100, 1600)

In [12]:
len(Q_)

1600

In [13]:
print(len(Q_))

1600


In [14]:
# Dis_q = Update_Dis(Dis_q,q,K_log)
# print(Dis_q)

# Loss Definition

In [15]:
loss_old = float('inf')
T_ = np.array(1e-04,ndmin=1)
Dis_q_ = 1


T = tf.placeholder(dtype='float32',shape=[1])
Dis_q = tf.placeholder(dtype='int32',shape=[])
Y = tf.placeholder(dtype='float32',shape=[Num_fine,counter//N_ref])


memAvg_dis_plot = []

New_q_2 = Update_Dis(Dis_q,q_val,K_log)
# Dis_q_ = Dis_q_*2

# T_ = np.array(1e-4,ndmin=1)
# Y = y_t 
# T = T_
dw_mat = dw(Y,Q_,q,T)
dw_mean = tf.reduce_mean( p_y_X*dw_mat *tf.reshape(q,[-1,1]))
MI = MI_f(dw_mat,p_y_X,T,q)
# p_y_X_ = p_y_X_f(dw_mat,q,T,Dis_q)
# q = q_k(p_y_X_) 

print(p_y_X)
New_p_y_X = p_y_X_f(dw_mat,q,T,p_y_X,Dis_q)
New_q = q_f(p_y_X,q_val)

New_q_3 = q_f(p_y_X,q_val,1e-2)

loss = F(q,dw_mat,T)

print(q)


loss_q = tf.reduce_sum((1+tf.sign(q-tf.constant(1e-2)/tf.to_float(Dis_q)))/2)


(64, 1, 1)
<tf.Variable 'Variable_1:0' shape=(64, 100) dtype=float32_ref>
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Tensor("Reshape_2:0", shape=(64,), dtype=float32)


In [16]:
loss_q = tf.reduce_sum((1+tf.sign(q-tf.constant(1e-2)/tf.to_float(Dis_q)))/2)

In [17]:
print(MI)

Tensor("Mean_1:0", shape=(), dtype=float32)


In [18]:
opt = tf.train.AdamOptimizer(learning_rate=0.001, name = 'Optimizer')
optimizer_sum =  tf.contrib.layers.optimize_loss(loss, tf.train.get_global_step(), 0.0099,opt,gradient_noise_scale=tf.sqrt(T)*1e-20  )

reset_optimizer_op = tf.variables_initializer(opt.variables())
sess = tf.Session()
sess.as_default()

# Initialization of Optimization 

In [19]:
sess.run(tf.global_variables_initializer())

Dis_q_ = 1

Index_training = [0,2,10,200, 400,600, 800,1000 ,1200,1400,1599]

T_ = np.array(1e5,ndmin=1)
alpha = 0.05

In [20]:
Avg_dis_plot = []
print('INDEX TRAINING:',Index_training)

start_time = datetime.datetime.now()

T_check = np.array(0.9 * T_ ,ndmin=1)

loss_old_ = float('inf')
q_ = sess.run( q,feed_dict={Y: y_t, Dis_q:Dis_q_ })
print('check1',q_)
print('# of Dis. prob.',sess.run( loss_q,feed_dict={Y: y_t, Dis_q:Dis_q_ }))
loss_,_ = sess.run([loss,New_q_2],feed_dict={Y: y_t ,T:T_, Dis_q:Dis_q_ })
Dis_q_ = Dis_q_ * 2
q_ = sess.run( q,feed_dict={Y: y_t, Dis_q:Dis_q_ })
print('# of Dis. prob.',sess.run( loss_q,feed_dict={Y: y_t, Dis_q:Dis_q_ }))
print('check2',q_)

print('check check',Dis_q_)
print(T_,'loss:',loss_ )

_  = sess.run([New_p_y_X],feed_dict={Y: y_t ,T:T_, Dis_q:Dis_q_ })
_  = sess.run([New_q],feed_dict={Y: y_t ,T:T_, Dis_q:Dis_q_ })

INDEX TRAINING: [0, 2, 10, 200, 400, 600, 800, 1000, 1200, 1400, 1599]
check1 [1.0000000e+00 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11
 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11
 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11
 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11
 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11
 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11
 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11
 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11
 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11
 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11
 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11
 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11
 1.5625003e-11 1.5625003e-11 1.5625003e-11 1.5625003e-11]
# of Dis. pr

In [21]:
save = list()

In [ ]:
save

[]

In [ ]:
while T_>1e-10:
    print('T:',T_)
    

    _  = sess.run([New_q_3],feed_dict={Y: y_t ,T:T_, Dis_q:Dis_q_ })
    _  = sess.run([New_p_y_X],feed_dict={Y: y_t ,T:T_, Dis_q:Dis_q_ })
    for itr in range (10001):
               
        _  = sess.run([optimizer_sum],feed_dict={Y: y_t ,T:T_, Dis_q:Dis_q_ })
        
        
        if itr % 10 == 0:
            
#             q_ = sess.run( q,feed_dict={Y: y_t, Dis_q:Dis_q_ })
#             print('check2',q_)
            _  = sess.run([New_p_y_X,New_q],feed_dict={Y: y_t ,T:T_, Dis_q:Dis_q_ })
            loss_,dw_check,MI_ = sess.run([loss,dw_mean,MI],feed_dict={Y: y_t ,T:T_, Dis_q:Dis_q_ })
            check = np.abs((loss_old_-loss_)/loss_)
            
            print(itr,'loss:',loss_,'dw:',dw_check,'MI:',MI_)
#             q_ = sess.run( q,feed_dict={Y: y_t, Dis_q:Dis_q_ })
#             print('check2',q_)
            
            Num_prob = sess.run( loss_q,feed_dict={Y: y_t, Dis_q:Dis_q_ })
            print('# of Dis. prob.',Num_prob)
            
            if check<0.0005 : 
                
                Avg_dis_plot.append([T_,dw_check ,MI_]) 
                break
            loss_old_ = loss_
            
    loss_old_ = float('inf')
    
    T_ = T_ * (1-alpha)
    
    if T_/T_check<1:
        T_check=T_check/10
        print(T_check)
        for ind in range(len(Index_training)):

            Q_0_, q_, K_loss_ = sess.run([Q_[Index_training[ind]], q,K_loss],feed_dict={Y: y_t, Dis_q:Dis_q_ })

            print(T_)
            plot(Q_0_,q_,y_t,Index_training[ind],T_,run_name)
            save.append([Q_0_,q_,y_t,Index_training[ind],T_,])
            
    Q_check = sess.run(Q_,feed_dict={Y: y_t })
    
    check2 = find_max_dic(Q_check,Dis_q_)
    
    print('check2:',check2)
    if check2>1e-13 and Dis_q_ < Num_coarse:
        
           
        _ = sess.run([New_q_2],feed_dict={Y: y_t ,T:T_, Dis_q:Dis_q_ })
        Dis_q_ = Dis_q_ * 2
        print(Dis_q_)
                      
    if Num_prob==1:
        
        Dis_q_ = 1
        _ = sess.run([New_q_2],feed_dict={Y: y_t ,T:T_, Dis_q:Dis_q_ })
        Dis_q_ = Dis_q_ * 2
        print(Dis_q_)

    sess.run(reset_optimizer_op)
end_time = datetime.datetime.now()
print(end_time-start_time)


T: [100000.]
0 loss: 0.016348077 dw: 12.771936 MI: 4.1081703e-06
# of Dis. prob. 2.0
10 loss: 0.01630476 dw: 12.738093 MI: 4.394189e-06
# of Dis. prob. 2.0
20 loss: 0.016265268 dw: 12.707241 MI: 4.7122658e-06
# of Dis. prob. 2.0
30 loss: 0.01623062 dw: 12.680173 MI: 5.041931e-06
# of Dis. prob. 2.0
40 loss: 0.016200583 dw: 12.656707 MI: 5.3553654e-06
# of Dis. prob. 2.0
50 loss: 0.016174665 dw: 12.636457 MI: 5.6465906e-06
# of Dis. prob. 2.0
60 loss: 0.016152326 dw: 12.619005 MI: 5.917515e-06
# of Dis. prob. 2.0
70 loss: 0.016133046 dw: 12.603942 MI: 6.168473e-06
# of Dis. prob. 2.0
80 loss: 0.016116384 dw: 12.590924 MI: 6.398986e-06
# of Dis. prob. 2.0
90 loss: 0.016101964 dw: 12.579659 MI: 6.6099938e-06
# of Dis. prob. 2.0
100 loss: 0.016089452 dw: 12.569885 MI: 6.803328e-06
# of Dis. prob. 2.0
110 loss: 0.016078599 dw: 12.561406 MI: 6.9819957e-06
# of Dis. prob. 2.0
120 loss: 0.01606915 dw: 12.554022 MI: 7.147703e-06
# of Dis. prob. 2.0
130 loss: 0.016060911 dw: 12.547586 MI: 7.3020

check2: 0.0
T: [14239.57413464]
0 loss: 0.11236285 dw: 12.499995 MI: 6.8914735e-05
# of Dis. prob. 2.0
10 loss: 0.112362914 dw: 12.5 MI: 6.8710426e-05
# of Dis. prob. 2.0
check2: 0.0
T: [13527.59542791]
0 loss: 0.11827668 dw: 12.499992 MI: 7.245902e-05
# of Dis. prob. 2.0
10 loss: 0.11827677 dw: 12.500003 MI: 7.222517e-05
# of Dis. prob. 2.0
check2: 0.0
T: [12851.21565651]
0 loss: 0.12450176 dw: 12.499992 MI: 7.618148e-05
# of Dis. prob. 2.0
10 loss: 0.12450186 dw: 12.500003 MI: 7.591499e-05
# of Dis. prob. 2.0
check2: 0.0
T: [12208.65487368]
0 loss: 0.13105449 dw: 12.499992 MI: 8.009484e-05
# of Dis. prob. 2.0
10 loss: 0.13105458 dw: 12.500001 MI: 7.9791236e-05
# of Dis. prob. 2.0
check2: 0.0
T: [11598.22213]
0 loss: 0.13795207 dw: 12.49999 MI: 8.4205254e-05
# of Dis. prob. 2.0
10 loss: 0.13795221 dw: 12.500005 MI: 8.386481e-05
# of Dis. prob. 2.0
check2: 0.0
T: [11018.3110235]
0 loss: 0.14521271 dw: 12.499991 MI: 8.852963e-05
# of Dis. prob. 2.0
10 loss: 0.14521284 dw: 12.500004 MI: 

check2: 0.0
T: [1490.51253825]
0 loss: 1.0734569 dw: 12.500006 MI: 0.00067155255
# of Dis. prob. 2.0
10 loss: 1.073456 dw: 12.5 MI: 0.0006536728
# of Dis. prob. 2.0
check2: 0.0
T: [1415.98691134]
0 loss: 1.1299548 dw: 12.500012 MI: 0.0007114882
# of Dis. prob. 2.0
10 loss: 1.1299537 dw: 12.499997 MI: 0.00069258804
# of Dis. prob. 2.0
check2: 0.0
T: [1345.18756577]
0 loss: 1.1894256 dw: 12.500006 MI: 0.0007546975
# of Dis. prob. 2.0
10 loss: 1.1894256 dw: 12.500005 MI: 0.00073528243
# of Dis. prob. 2.0
check2: 0.0
T: [1277.92818748]
0 loss: 1.252027 dw: 12.500005 MI: 0.00080200227
# of Dis. prob. 2.0
10 loss: 1.2520275 dw: 12.500011 MI: 0.0007828136
# of Dis. prob. 2.0
check2: 0.0
T: [1214.03177811]
0 loss: 1.317923 dw: 12.500003 MI: 0.00085441006
# of Dis. prob. 2.0
10 loss: 1.3179231 dw: 12.500003 MI: 0.0008361575
# of Dis. prob. 2.0
check2: 0.0
T: [1153.3301892]
0 loss: 1.3872873 dw: 12.500003 MI: 0.0009131136
# of Dis. prob. 2.0
10 loss: 1.387288 dw: 12.500008 MI: 0.0008962281
# of 

80 loss: 8.983086 dw: 12.614075 MI: 0.06295149
# of Dis. prob. 2.0
90 loss: 8.941652 dw: 12.624903 MI: 0.06611894
# of Dis. prob. 2.0
100 loss: 8.89598 dw: 12.633188 MI: 0.0692544
# of Dis. prob. 2.0
110 loss: 8.8449745 dw: 12.639242 MI: 0.07233692
# of Dis. prob. 2.0
120 loss: 8.78947 dw: 12.643467 MI: 0.07530794
# of Dis. prob. 2.0
130 loss: 8.732504 dw: 12.646294 MI: 0.078129426
# of Dis. prob. 2.0
140 loss: 8.676214 dw: 12.648101 MI: 0.080848366
# of Dis. prob. 2.0
150 loss: 8.61861 dw: 12.649204 MI: 0.08359312
# of Dis. prob. 2.0
160 loss: 8.553642 dw: 12.649849 MI: 0.08638393
# of Dis. prob. 2.0
170 loss: 8.481933 dw: 12.650222 MI: 0.089064226
# of Dis. prob. 2.0
180 loss: 8.411058 dw: 12.650435 MI: 0.0915127
# of Dis. prob. 2.0
190 loss: 8.343669 dw: 12.650603 MI: 0.09371937
# of Dis. prob. 2.0
200 loss: 8.279569 dw: 12.650805 MI: 0.09570981
# of Dis. prob. 2.0
210 loss: 8.218085 dw: 12.651124 MI: 0.097522065
# of Dis. prob. 2.0
220 loss: 8.158551 dw: 12.651691 MI: 0.099190466
#

1290 loss: 7.653622 dw: nan MI: nan
# of Dis. prob. 2.0
1300 loss: 7.5095134 dw: nan MI: nan
# of Dis. prob. 2.0
1310 loss: 7.3732677 dw: nan MI: nan
# of Dis. prob. 2.0
1320 loss: 7.2443986 dw: nan MI: nan
# of Dis. prob. 2.0
1330 loss: 7.1223497 dw: nan MI: nan
# of Dis. prob. 2.0
1340 loss: 7.0065074 dw: nan MI: nan
# of Dis. prob. 2.0
1350 loss: 6.896324 dw: nan MI: nan
# of Dis. prob. 2.0
1360 loss: 6.7913184 dw: nan MI: nan
# of Dis. prob. 2.0
1370 loss: 6.6910954 dw: nan MI: nan
# of Dis. prob. 2.0
1380 loss: 6.5953326 dw: nan MI: nan
# of Dis. prob. 2.0
1390 loss: 6.5037875 dw: nan MI: nan
# of Dis. prob. 2.0
1400 loss: 6.4162645 dw: nan MI: nan
# of Dis. prob. 2.0
1410 loss: 6.332605 dw: nan MI: nan
# of Dis. prob. 2.0
1420 loss: 6.252671 dw: nan MI: nan
# of Dis. prob. 2.0
1430 loss: 6.176318 dw: nan MI: nan
# of Dis. prob. 2.0
1440 loss: 6.1033726 dw: nan MI: nan
# of Dis. prob. 2.0
1450 loss: 6.0335655 dw: nan MI: nan
# of Dis. prob. 2.0
1460 loss: 5.966398 dw: nan MI: nan


...

4370 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4380 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4390 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4400 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4410 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4420 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4430 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4440 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4450 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4460 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4470 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4480 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4490 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4500 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4510 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4520 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4530 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4540 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4550 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
4560 loss: n

5950 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
5960 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
5970 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
5980 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
5990 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
6000 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
6010 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
6020 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
6030 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
6040 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
6050 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
6060 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
6070 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
6080 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
6090 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
6100 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
6110 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
6120 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
6130 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
6140 loss: n

7530 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7540 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7550 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7560 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7570 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7580 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7590 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7600 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7610 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7620 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7630 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7640 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7650 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7660 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7670 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7680 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7690 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7700 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7710 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
7720 loss: n

9110 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9120 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9130 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9140 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9150 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9160 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9170 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9180 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9190 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9200 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9210 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9220 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9230 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9240 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9250 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9260 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9270 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9280 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9290 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0
9300 loss: n

690 loss: nan dw: nan MI: nan
# of Dis. prob. 32.0


In [ ]:
np.array(y_t).max()

In [ ]:

Dis_q_ = 1
_ = sess.run([New_q_2],feed_dict={Y: y_t ,T:T_, Dis_q:Dis_q_ })
Dis_q_ = Dis_q_ * 2
print(Dis_q_)
Num_prob = sess.run( loss_q,feed_dict={Y: y_t, Dis_q:Dis_q_ })
print(Num_prob)

In [ ]:
i = 0
print(Dis_q_)
print(np.resize(np.sort(Q_check[i]),[-1])[0:Dis_q_])
Q_srt = np.resize(np.sort(Q_check[i]),[-1])[0:Dis_q_]

print(Q_srt.shape)

In [ ]:
len(Index_training)

In [ ]:
for ind in range(len(Index_training)):
    ind = 9
    Q_0_, q_, K_loss_ = sess.run([Q_[Index_training[ind]], q,K_loss],feed_dict={Y: y_t, Dis_q:Dis_q_ })

    print(T_)
    plot(Q_0_,q_,y_t,Index_training[ind],T_,'check')

print(q_)

In [ ]:
[run_name+'check']

In [ ]:
A = np.asarray(Avg_dis_plot)

B = np.ndarray.tolist(A[:,2])
dw = np.ndarray.tolist(A[:,1])
ind_B = np.isnan(B)

B = [B[i] for i, x in enumerate(ind_B) if x==False]
dw = [dw[i] for i, x in enumerate(ind_B) if x==False]
name_MI = './'+run_name+'/Mutual Info'

plt.plot(dw,B);

plt.xscale('log')
plt.yscale('linear')
plt.ylabel('Mutual Information')
plt.xlabel('Average Distortion')
plt.grid(True)


plt.savefig(name_MI, dpi=150)
plt.close()

In [ ]:

plt.plot(dw,B);

plt.xscale('log')
plt.yscale('linear')
plt.ylabel('Mutual Information')
plt.xlabel('Average Distortion')
plt.grid(True)

In [ ]:
len(save)

In [ ]:
import pickle

In [ ]:
fp

In [ ]:
with open("test.txt", "wb") as fp:   #Pickli
    pickle.dump(save, fp)

In [ ]:
with open("test.txt", "rb") as fp:   # Unpickling
...   b = pickle.load(fp)
... 

In [ ]:
np.sum(K_loss_[-400][:,0,0]*q_)

In [ ]:
plt.plot([np.sum(k[:,0,0]*q_)/1e-5 for k in K_loss_])

In [ ]:
import pandas as pd

In [ ]:
np.array([np.sum(k[:,0,0]*q_) for k in K_loss_])

In [ ]:
np.mean(y_t,axis=0).shape

In [ ]:
plt.plot([i/1600 for i in range(0,1600)],np.mean(y_t,axis=0)/np.array([np.sum(k[:,0,0]*q_) for k in K_loss_]))

# plt.ylabel(r'$\dfrac{q}{K_eff}$')
plt.xlabel(r'Time')
plt.ylabel(r'$\frac{<q>}{<K_eff>}$')
# plt.ylim(0,0.03)
# plt.xlim(0,1)
plt.savefig(f'{run_name}/q_keff', dpi=150)


In [ ]:
np.cumsum(q_[indx])

In [ ]:
indx = [np.argsort(np.array(K_loss_[1599][:,0,0]).T,axis=0)]

pd.DataFrame(np.array([K_loss_[1599][:,0,0][indx],np.cumsum(q_[indx])]).T)


np.interp(0.1,np.cumsum(q_[indx]),K_loss_[1599][:,0,0][indx])



In [ ]:
def percenitile_of(K_loss_,perc):
    indx_sort = lambda  k :  [np.argsort(np.array(k[:,0,0]).T,axis=0)]

    return [np.interp(perc,np.cumsum(q_[indx_sort(k)]),k[:,0,0][indx_sort(k)]) for k in K_loss_]
    
    

In [ ]:
percentiles = pd.DataFrame([percenitile_of(K_loss_,perc/10) for perc in range(0,11)])

In [ ]:
percentiles.to_csv('percentile_slow_var.csv')

In [ ]:
A.shape

In [ ]:
import matplotlib
from matplotlib import rc

In [ ]:
# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

In [ ]:
plt.scatter(A[:,1],A[:,2],c=A[:,0],norm=matplotlib.colors.LogNorm())

plt.xscale('log')
plt.yscale('linear')
plt.ylabel(r'Mutual Information')
plt.xlabel('Average Distortion')
plt.grid(True)
# plt.xlim(2e-3,8e0)
# plt.ylim(0,0.035)
plt.colorbar()

plt.savefig(f'{run_name}/Mutual_info_', dpi=150)


In [ ]:
np.log(5)

In [ ]:
[a[0] for a in A[:,0]]

In [ ]:
plt.plot([np.sum(k[:,0,0]*q_/2) for k in K_loss_])

In [ ]:
len(K_loss_)

In [ ]:
q_

In [ ]:
df_K = pd.DataFrame(np.stack(K_loss_)[:,:,0,0])

df_K.to_pickle('./df_K_2.pkl')

In [ ]:
df_q = pd.DataFrame(np.stack(q_))

df_q.to_pickle('./df_q_2.pkl')